In [13]:
!pip install transformers
!pip install torch
!pip install datasets
!pip install huggingface_hub
!pip install tf-keras
!pip install accelerate -U
!pip install transformers[torch]
!pip install evaluate


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from tqdm import tqdm

In [2]:
from huggingface_hub import login

# Login using your Hugging Face token

Saessak_token = ""

login(token=Saessak_token)

c:\Users\PHY\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\PHY\.cache\huggingface\token
Login successful


In [1]:
# 여기서 난 오류로 data_path 파악
from datasets import load_dataset, load_metric, Dataset

dataset = load_dataset("Saessak/Conversation", cache_dir="./hf_cache")

ModuleNotFoundError: No module named 'datasets'

In [4]:
import pandas as pd
import pyarrow.parquet as pq

# Load the dataset file manually using pyarrow

# 데이터 패스는 매번 변할 것이다.
# load_dataset 오류를 보면 어떤 패스인지 나온
data_path = rf""
table = pq.read_table(data_path)

# Convert to pandas DataFrame
df = table.to_pandas()

# Display the current column names
print("Current column names:", df.columns)

# Define the expected column names mapping
expected_columns = {
    '음성인식결과': 'speech_recognition_result',
    '스크립트ID': 'script_id',
    '파일명': 'file_name',
    '녹음시간': 'recording_time',
    '녹음품질': 'recording_quality',
    '녹음일시': 'recording_datetime',
    '스크립트셋번호': 'script_set_number',
    '녹음환경': 'recording_environment',
    '수집방법': 'collection_method',
    '지역': 'region',
    '녹음도구': 'recording_tool',
    '대화주제': 'conversation_topic',
    '성별': 'gender',
    '녹음자ID': 'recorder_id',
    '나이': 'age'
}

# Rename the columns
df.rename(columns=expected_columns, inplace=True)

# Save the DataFrame back to Parquet with the new column names
new_data_path = './hf_cache/renamed_dataset.parquet'
df.to_parquet(new_data_path)

print("Renamed column names:", df.columns)


Current column names: Index(['음성인식결과', '스크립트ID', '파일명', '녹음시간', '녹음품질', '녹음일시', '스크립트셋번호', '녹음환경',
       '수집방법', '지역', '녹음도구', '대화주제', '성별', '녹음자ID', '나이'],
      dtype='object')
Renamed column names: Index(['speech_recognition_result', 'script_id', 'file_name', 'recording_time',
       'recording_quality', 'recording_datetime', 'script_set_number',
       'recording_environment', 'collection_method', 'region',
       'recording_tool', 'conversation_topic', 'gender', 'recorder_id', 'age'],
      dtype='object')


In [5]:
# Use a different cache directory
dataset = load_dataset('parquet', data_files=new_data_path, cache_dir="./hf_cache")

# Display the first few rows of the training set
print(dataset['train'].to_pandas().head())

Generating train split: 2509153 examples [00:00, 3698231.72 examples/s]


  speech_recognition_result   script_id  \
0                 내일 또 얘기하자  소아남녀-64059   
1                교복을 입고 왔어요  소아남녀-64307   
2                  나는 씩씩해요   소아남녀-02094   
3           빨리 내일이 됐으면 좋겠다   소아남녀-03402   
4    (NO:그럼 물고기똥도) 같이 맞는 거야  소아남녀-02634   

                                         file_name  recording_time  \
0  소아남여_소아남여8001_F_1611226664-0_5_수도권_실외_64059.wav            2.73   
1  소아남여_소아남여8001_F_1611226664-0_5_수도권_실외_64307.wav            3.50   
2    소아남여_소아남여02_F_1565719340-0_7_수도권_실내_02094.wav            2.39   
3    소아남여_소아남여02_F_1565719340-0_7_수도권_실내_03402.wav            4.01   
4    소아남여_소아남여02_F_1565719340-0_7_수도권_실내_02634.wav            3.50   

  recording_quality   recording_datetime script_set_number  \
0               16K  2021-01-30 14:45:44       I_소아남녀_8001   
1               16K  2021-02-04 13:31:00       I_소아남녀_8001   
2               16K  2020-12-31 16:30:42          T_소아남녀_2   
3               16K  2021-01-06 18:56:32          T_소아남녀_2   
4         

In [6]:
df_age_conv = dataset['train'].to_pandas().loc[:,['speech_recognition_result','age']]

In [7]:
df_age_conv.rename(columns = {'speech_recognition_result':'speech'},inplace = True)
df_age_conv

,speech,age
0,내일 또 얘기하자,5
1,교복을 입고 왔어요,5
2,나는 씩씩해요,7
3,빨리 내일이 됐으면 좋겠다,7
4,(NO:그럼 물고기똥도) 같이 맞는 거야,7
...,...,...
2509148,오늘 병원 가서 키 재고 몸무게 재고 시력 검사도 했어요,7
2509149,어제 인기가요 봤어,7
2509150,할머니 진지 드세요,7
2509151,떡볶이가 너무 매워요,7


# 동일한 개수의 text

In [8]:
# Rename columns if necessary to match expected format
df_age_conv.columns = ['text', 'label']

original_labels = df_age_conv['label'].unique()
label_mapping = {original_label: new_label for new_label, original_label in enumerate(sorted(original_labels))}

# Apply the mapping to the 'label' column
df_age_conv['label'] = df_age_conv['label'].map(label_mapping)


label_counts = df_age_conv['label'].value_counts()

# Determine the minimum count of texts for any label
min_count = label_counts.min()

# Sample texts from each label
balanced_df = df_age_conv.groupby('label').apply(lambda x: x.sample(min_count)).reset_index(drop=True)

# Optionally, you can check the counts to verify
print(balanced_df['label'].value_counts())

balanced_df

label
0    24062
1    24062
2    24062
3    24062
4    24062
5    24062
6    24062
7    24062
Name: count, dtype: int64


C:\Users\PHY\AppData\Local\Temp\ipykernel_16952\3665979292.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df = df_age_conv.groupby('label').apply(lambda x: x.sample(min_count)).reset_index(drop=True)


,text,label
0,이 옷은 편해요,0
1,놀이터는 재미있어,0
2,영화에도 나왔어요,0
3,고개를 숙여요,0
4,옥수수를 반으로 잘랐어요,0
...,...,...
192491,내 동생은 나보다 걔가 좋은가 봐,7
192492,그럼 나 안 먹을 거야,7
192493,불이 나면 숨어 있으면 안된댔어,7
192494,(SP:아)침 일찍 공항에 가서 비행기를 타고 제주도에 갔어,7


# 모델 튜닝

In [9]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")


CUDA is available. Using GPU.


In [10]:
import os

output_dir = rf"C:\Users\PHY\Desktop\robert\result"
logging_dir = rf"C:\Users\PHY\Desktop\robert\log"

output_dir_sonnet = rf"C:\Users\PHY\Desktop\robert\result_sonnet"
logging_dir_sonnet = rf"C:\Users\PHY\Desktop\robert\log_sonnet"

# Medium 참고 코드 

## Dataset으로 다시 변환

In [11]:
from datasets import Dataset

balanced_dataset = Dataset.from_pandas(balanced_df)

balanced_dataset

splitted_datasets = balanced_dataset.train_test_split(test_size = 0.3)

splitted_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 134747
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 57749
    })
})

In [12]:
train_df = pd.DataFrame(splitted_datasets['train'])
test_df = pd.DataFrame(splitted_datasets['test'])

print(train_df['label'].value_counts())
print(test_df['label'].value_counts())

label
6    16957
7    16946
3    16924
2    16871
5    16827
4    16793
1    16775
0    16654
Name: count, dtype: int64
label
0    7408
1    7287
4    7269
5    7235
2    7191
3    7138
7    7116
6    7105
Name: count, dtype: int64


In [13]:
import evaluate

metric = evaluate.load("accuracy")

metric.compute(predictions=[0,0,1,1], references=[0,1,1,1])

{'accuracy': 0.75}

In [14]:
from transformers import AutoTokenizer

model_checkpoint = "klue/roberta-large"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast = True)

tokenizer(["안녕, 나는 문장이야!"])

{'input_ids': [[0, 5891, 16, 717, 2259, 6265, 2052, 2275, 5, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [15]:
def preprocess_function_batch(examples):
    return tokenizer(examples["text"], truncation=True)

splitted_datasets_encoded = splitted_datasets.map(preprocess_function_batch, batched=True)

Map: 100%|██████████| 57749/57749 [00:00<00:00, 60359.04 examples/s]


In [16]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=8)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#!pip install tensorboard

model_output_dir = rf"C:\Users\PHY\Desktop\robert\klueroberta_large_finetuned_age_classification"


print(model_output_dir)

# 모니터링 위해 TensorBoard 시작
%reload_ext tensorboard
%tensorboard --logdir '{model_output_dir}'/runs

C:\Users\PHY\Desktop\robert\klueroberta_large_finetuned_age_classification


Reusing TensorBoard on port 6008 (pid 9296), started 10:50:12 ago. (Use '!kill 9296' to kill it.)

## Training 파라미터 지정

In [18]:
args = TrainingArguments(
    # output_dir: directory where the model checkpoints will be saved.
    output_dir=model_output_dir,
    # evaluation_strategy (default "no"):
    # Possible values are:
    # "no": No evaluation is done during training.
    # "steps": Evaluation is done (and logged) every eval_steps.
    # "epoch": Evaluation is done at the end of each epoch.
    eval_strategy="steps",
    # eval_steps: Number of update steps between two evaluations if
    # evaluation_strategy="steps". Will default to the same value as
    # logging_steps if not set.
    eval_steps=50,
    # logging_strategy (default: "steps"): The logging strategy to adopt during
    # training (used to log training loss for example). Possible values are:
    # "no": No logging is done during training.
    # "epoch": Logging is done at the end of each epoch.
    # "steps": Logging is done every logging_steps.
    logging_strategy="steps",
    # logging_steps (default 500): Number of update steps between two logs if
    # logging_strategy="steps".
    logging_steps=500,
    # save_strategy (default "steps"):
    # The checkpoint save strategy to adopt during training. Possible values are:
    # "no": No save is done during training.
    # "epoch": Save is done at the end of each epoch.
    # "steps": Save is done every save_steps (default 500).
    save_strategy="steps",
    # save_steps (default: 500): Number of updates steps before two checkpoint
    # saves if save_strategy="steps".
    save_steps=2000,
    # learning_rate (default 5e-5): The initial learning rate for AdamW optimizer.
    # Adam algorithm with weight decay fix as introduced in the paper
    # Decoupled Weight Decay Regularization.
    learning_rate=0.001,
    # per_device_train_batch_size: The batch size per GPU/TPU core/CPU for training.

    # 32 일때 전용 GPU 메모리 10GB 밖에 못쓰고 있음!!!
    per_device_train_batch_size=64,
    # per_device_eval_batch_size: The batch size per GPU/TPU core/CPU for evaluation.
    per_device_eval_batch_size=64,
    # num_train_epochs (default 3.0): Total number of training epochs to perform
    # (if not an integer, will perform the decimal part percents of the last epoch
    # before stopping training).
    num_train_epochs=3,
    # load_best_model_at_end (default False): Whether or not to load the best model
    # found during training at the end of training.
    load_best_model_at_end=True,
    # metric_for_best_model:
    # Use in conjunction with load_best_model_at_end to specify the metric to use
    # to compare two different models. Must be the name of a metric returned by
    # the evaluation with or without the prefix "eval_".
    metric_for_best_model="accuracy",
    # report_to:
    # The list of integrations to report the results and logs to. Supported
    # platforms are "azure_ml", "comet_ml", "mlflow", "tensorboard" and "wandb".
    # Use "all" to report to all integrations installed, "none" for no integrations.
    report_to="tensorboard"
)

## 모델 생성 및 train

In [19]:
import numpy as np

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels = 8)

def compute_metrics(eval_pred):

    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis=1)

    results = metric.compute(predictions=predictions, references=labels)

    return results

trainer = Trainer(
    model_init = model_init,
    args=args,
    train_dataset=splitted_datasets_encoded["train"],
    eval_dataset=splitted_datasets_encoded["test"].shuffle(42).select(range(150)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)  

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
                                                 
  1%|          | 52/6318 [00:05<12:48,  8.15it/s]

{'eval_loss': 2.1396846771240234, 'eval_accuracy': 0.14666666666666667, 'eval_runtime': 0.1002, 'eval_samples_per_second': 1496.651, 'eval_steps_per_second': 29.933, 'epoch': 0.02}


                                                  
  2%|▏         | 102/6318 [00:10<12:53,  8.04it/s]

{'eval_loss': 2.1986098289489746, 'eval_accuracy': 0.14, 'eval_runtime': 0.1056, 'eval_samples_per_second': 1420.517, 'eval_steps_per_second': 28.41, 'epoch': 0.05}


                                                  
  2%|▏         | 151/6318 [00:16<13:42,  7.49it/s]

...

                                                   
 42%|████▏     | 2651/6318 [04:52<07:28,  8.17it/s]

{'eval_loss': 2.0974044799804688, 'eval_accuracy': 0.1, 'eval_runtime': 0.094, 'eval_samples_per_second': 1595.84, 'eval_steps_per_second': 31.917, 'epoch': 1.26}


                                                   
 43%|████▎     | 2701/6318 [04:58<07:34,  7.95it/s]

{'eval_loss': 2.0995752811431885, 'eval_accuracy': 0.10666666666666667, 'eval_runtime': 0.0931, 'eval_samples_per_second': 1611.896, 'eval_steps_per_second': 32.238, 'epoch': 1.28}


 43%|████▎     | 2728/6318 [05:01<06:24,  9.33it/s]

# Test 정확도 확인

In [ ]:
dataset_test_encoded = splitted_datasets["test"].map(preprocess_function_batch, batched = True)

test_predictions = trainer.predict(dataset_test_encoded)

test_predictions_argmax = np.argmax(test_predictions[0], axis=1)

test_references = np.array(splitted_datasets["test"]["label"])

metric.compute(predictions=test_predictions_argmax, references=test_references)

Map: 100%|██████████| 57749/57749 [00:01<00:00, 54573.32 examples/s]


{'accuracy': 0.12580304420855773}

In [ ]:
#!pip install ipython

# Create a DataFrame for visualization
test_texts = splitted_datasets["test"]["text"]
comparison_df = pd.DataFrame({
    'text': test_texts,
    'actual_label': test_references,
    'predicted_label': test_predictions_argmax
})
comparison_df['correct'] = comparison_df['actual_label'] == comparison_df['predicted_label']

# Display the comparison DataFrame
import IPython.display as display
display.display(comparison_df)

,text,actual_label,predicted_label,correct
0,아니 유치원 다닐 때가 좋아,7,7,True
1,(NO:집 못 찾아갈 것) 같아 (NO:),6,7,False
2,비가 주룩주룩 내려,4,7,False
3,빨리 출발해요,0,7,False
4,다 풀었어요,2,7,False
...,...,...,...,...
57744,별처럼 생긴 건 뭐야,0,7,False
57745,손톱 칠해드릴게요,5,7,False
57746,여기 있네,2,7,False
57747,토스트를 한 입 먹어요,2,7,False
